# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    
    # By setting (keeping) num_words at None, the Tokenizer will
    # use all words available in the input text. If we want to restrict
    # it to the top N words, we would use num_words=N here.
    tokenizer = Tokenizer(num_words=None, char_level=False)
    tokenizer.fit_on_texts(x)
    
    return tokenizer.texts_to_sequences(x), tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'a': 3, 'lazy': 8, 'won': 16, 'quick': 2, 'sentence': 21, 'fox': 5, 'brown': 4, 'lexicography': 15, 'jove': 11, 'this': 18, 'the': 1, 'my': 12, 'prize': 17, 'study': 13, 'short': 20, 'over': 7, 'dog': 9, 'by': 10, 'is': 19, 'of': 14, 'jumps': 6}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [11]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # Pad at the end, but should a sentence be longer than the specified
    # length, truncate from the beginning. Sutskever et al. describe in
    # "Sequence to Sequence Learning with Neural Networks" that feeding the
    # sentence to the network in reverse introduces meaningful short-term
    # dependencies that help in training a recurrent network. Keeping the
    # end of the sentence rather than the beginning is our best approximation
    # here. This assumes we're processing romanic languages.
    return pad_sequences(x, maxlen=length, padding='post', truncating='pre')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [12]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.

- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.

### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gap between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)

Note to self: The image appears to be a bit misleading because the `brown` position of the English text corresponds with the `renard` position of the French text. This is technically correct; `renard`, however, means `fox`. As [Wikipedia](https://fr.wikipedia.org/wiki/The_quick_brown_fox_jumps_over_the_lazy_dog) gives away, `The quick brown fox jumps over the lazy dog` translates to `Le vif renard brun saute par-dessus le chien paresseux` - this sentence is tailored to contain all letters of the alphabet, but it shows that the `the brown fox` is `le renard brun` indeed.

![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [37]:
print('English vocabulary size:', len(english_tokenizer.word_index))
print('French vocabulary size:', len(french_tokenizer.word_index))

English vocabulary size: 199
French vocabulary size: 344


In [53]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, 
                 learning_rate: float=0.001):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    inputs = Input(shape=input_shape[1:])
    
    recurrent = GRU(units=output_sequence_length, return_sequences=True, activation=None)(inputs)
    recurrent = PReLU()(recurrent)

    # Each of the `output_sequence_length` output time steps should
    # be a vector of `max(1024, 2*french_vocab_size)` dimensions.
    dense = TimeDistributed(Dense(max(1024, 2*french_vocab_size), activation=None))(recurrent)
    dense = PReLU()(dense)
    
    # Not applying dropout because the network doesn't appear to
    # overfit in the given scenario.
        
    # Each of the `output_sequence_length` output time steps should
    # be a vector of `french_vocab_size` dimensions.
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s - loss: 2.3885 - acc: 0.5252 - val_loss: nan - val_acc: 0.6004
Epoch 2/10
110288/110288 [==============================] - 7s - loss: 1.5002 - acc: 0.6130 - val_loss: nan - val_acc: 0.6199
Epoch 3/10
110288/110288 [==============================] - 7s - loss: 1.3766 - acc: 0.6336 - val_loss: nan - val_acc: 0.6434
Epoch 4/10
110288/110288 [==============================] - 7s - loss: 1.3118 - acc: 0.6496 - val_loss: nan - val_acc: 0.6563
Epoch 5/10
110288/110288 [==============================] - 7s - loss: 1.2617 - acc: 0.6600 - val_loss: nan - val_acc: 0.6643
Epoch 6/10
110288/110288 [==============================] - 7s - loss: 1.2206 - acc: 0.6705 - val_loss: nan - val_acc: 0.6800
Epoch 7/10
110288/110288 [==============================] - 7s - loss: 1.1865 - acc: 0.6802 - val_loss: nan - val_acc: 0.6859
Epoch 8/10
110288/110288 [==============================] - 7s - lo

That's 70% accuracy on the validation set. Here's the constructed network.

In [119]:
simple_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
gru_56 (GRU)                 (None, 21, 21)            1449      
_________________________________________________________________
p_re_lu_31 (PReLU)           (None, 21, 21)            441       
_________________________________________________________________
time_distributed_80 (TimeDis (None, 21, 1024)          22528     
_________________________________________________________________
p_re_lu_32 (PReLU)           (None, 21, 1024)          21504     
_________________________________________________________________
time_distributed_81 (TimeDis (None, 21, 344)           352600    
Total params: 398,522
Trainable params: 398,522
Non-trainable params: 0
_________________________________________________________________


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [140]:
from keras.layers.embeddings import Embedding
from keras.layers import Reshape


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size, 
                 learning_rate: float=0.001, english_embedding_size: int=256):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    inputs = Input(shape=input_shape[1:])
    
    embed_english = Embedding(english_vocab_size, english_embedding_size, input_length=input_shape[1])(inputs)
    
    recurrent = GRU(units=output_sequence_length, return_sequences=True, activation=None)(embed_english)
    recurrent = PReLU()(recurrent)

    dense = TimeDistributed(Dense(max(1024, 2*french_vocab_size), activation=None))(recurrent)
    dense = PReLU()(dense)
    
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# The unit test for test_embed_model() differs from the other ones
# in the shape of the input. I'm skipping the reshape in the implementation below.
tests.test_embed_model(embed_model)


# Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# Train the neural network
embedding_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
embedding_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embedding_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s - loss: 2.8404 - acc: 0.5057 - val_loss: nan - val_acc: 0.6438
Epoch 2/10
110288/110288 [==============================] - 8s - loss: 1.0909 - acc: 0.7025 - val_loss: nan - val_acc: 0.7467
Epoch 3/10
110288/110288 [==============================] - 8s - loss: 0.7331 - acc: 0.7819 - val_loss: nan - val_acc: 0.8053
Epoch 4/10
110288/110288 [==============================] - 8s - loss: 0.5936 - acc: 0.8158 - val_loss: nan - val_acc: 0.8266
Epoch 5/10
110288/110288 [==============================] - 8s - loss: 0.5225 - acc: 0.8350 - val_loss: nan - val_acc: 0.8434
Epoch 6/10
110288/110288 [==============================] - 8s - loss: 0.4757 - acc: 0.8474 - val_loss: nan - val_acc: 0.8531
Epoch 7/10
110288/110288 [==============================] - 8s - loss: 0.4406 - acc: 0.8569 - val_loss: nan - val_acc: 0.8599
Epoch 8/10
110288/110288 [==============================] - 8s - l

With the embedding layer we easily hit 87% accuracy on the validation set. Here's the constructed network:

In [117]:
embedding_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_98 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_37 (Embedding)     (None, 21, 256)           50944     
_________________________________________________________________
gru_92 (GRU)                 (None, 21, 21)            17514     
_________________________________________________________________
p_re_lu_67 (PReLU)           (None, 21, 21)            441       
_________________________________________________________________
time_distributed_116 (TimeDi (None, 21, 1024)          22528     
_________________________________________________________________
p_re_lu_68 (PReLU)           (None, 21, 1024)          21504     
_________________________________________________________________
time_distributed_117 (TimeDi (None, 21, 344)           352600    
Total para

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [132]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size,
            learning_rate: float = 0.01):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    inputs = Input(shape=input_shape[1:])
    
    recurrent = GRU(units=output_sequence_length, return_sequences=True, activation=None)   
    bidirectional = Bidirectional(recurrent, input_shape=input_shape[1:])(inputs)
    bidirectional = PReLU()(bidirectional)
    
    dense = TimeDistributed(Dense(max(1024, 2*french_vocab_size), activation=None))(bidirectional)
    dense = PReLU()(dense)
    
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(dense)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)


# Reshaping the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bidirectional_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
bidirectional_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 11s - loss: 1.5743 - acc: 0.6127 - val_loss: nan - val_acc: 0.6623
Epoch 2/10
110288/110288 [==============================] - 9s - loss: 1.0768 - acc: 0.6772 - val_loss: nan - val_acc: 0.6959
Epoch 3/10
110288/110288 [==============================] - 9s - loss: 0.9777 - acc: 0.7005 - val_loss: nan - val_acc: 0.6990
Epoch 4/10
110288/110288 [==============================] - 9s - loss: 0.9246 - acc: 0.7106 - val_loss: nan - val_acc: 0.7114
Epoch 5/10
110288/110288 [==============================] - 9s - loss: 0.9054 - acc: 0.7139 - val_loss: nan - val_acc: 0.7190
Epoch 6/10
110288/110288 [==============================] - 9s - loss: 0.8674 - acc: 0.7207 - val_loss: nan - val_acc: 0.7228
Epoch 7/10
110288/110288 [==============================] - 9s - loss: 0.8423 - acc: 0.7256 - val_loss: nan - val_acc: 0.7312
Epoch 8/10
110288/110288 [==============================] - 9s - l

With 73% accuracy on the validation set, this approach is slightly better than the plain forward-only RNN, but still much worse than the network using embeddings. Here's the layout:

In [133]:
bidirectional_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_114 (InputLayer)       (None, 21, 1)             0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 21, 42)            2898      
_________________________________________________________________
p_re_lu_84 (PReLU)           (None, 21, 42)            882       
_________________________________________________________________
time_distributed_130 (TimeDi (None, 21, 1024)          44032     
_________________________________________________________________
p_re_lu_85 (PReLU)           (None, 21, 1024)          21504     
_________________________________________________________________
time_distributed_131 (TimeDi (None, 21, 344)           352600    
Total params: 421,916
Trainable params: 421,916
Non-trainable params: 0
_________________________________________________________________


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [138]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size,
                learning_rate: float=0.01):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    inputs = Input(shape=input_shape[1:])
    
    recurrent = GRU(units=output_sequence_length, return_sequences=False, activation=None)(inputs)
    recurrent = PReLU()(recurrent)

    repeated = RepeatVector(output_sequence_length)(recurrent)
    
    recurrent = GRU(units=output_sequence_length, return_sequences=True, activation=None)(repeated)
    recurrent = PReLU()(recurrent)
    
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(recurrent)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    
    return None
tests.test_encdec_model(encdec_model)


# Reshaping the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encoderdecoder_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))
encoderdecoder_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encoderdecoder_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s - loss: 2.2135 - acc: 0.5087 - val_loss: nan - val_acc: 0.5389
Epoch 2/10
110288/110288 [==============================] - 7s - loss: 1.5672 - acc: 0.5818 - val_loss: nan - val_acc: 0.5980
Epoch 3/10
110288/110288 [==============================] - 7s - loss: 1.4252 - acc: 0.6071 - val_loss: nan - val_acc: 0.6048
Epoch 4/10
110288/110288 [==============================] - 7s - loss: 1.3786 - acc: 0.6176 - val_loss: nan - val_acc: 0.6232
Epoch 5/10
110288/110288 [==============================] - 7s - loss: 1.3462 - acc: 0.6246 - val_loss: nan - val_acc: 0.6242
Epoch 6/10
110288/110288 [==============================] - 7s - loss: 1.3231 - acc: 0.6296 - val_loss: nan - val_acc: 0.6253
Epoch 7/10
110288/110288 [==============================] - 7s - loss: 1.2769 - acc: 0.6395 - val_loss: nan - val_acc: 0.6462
Epoch 8/10
110288/110288 [==============================] - 7s - lo

About 65% accuracy on the validation set. Here's the layout:

In [139]:
encoderdecoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_121 (InputLayer)       (None, 21, 1)             0         
_________________________________________________________________
gru_120 (GRU)                (None, 21)                1449      
_________________________________________________________________
p_re_lu_98 (PReLU)           (None, 21)                21        
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 21, 21)            0         
_________________________________________________________________
gru_121 (GRU)                (None, 21, 21)            2709      
_________________________________________________________________
p_re_lu_99 (PReLU)           (None, 21, 21)            441       
_________________________________________________________________
time_distributed_138 (TimeDi (None, 21, 344)           7568      
Total para

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [242]:
from keras.layers import Reshape
from keras.layers.advanced_activations import LeakyReLU

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size,
               learning_rate: float=0.001, english_embedding_size: int=512,
               hidden_layer_length: int=128, out_sequence_length: int=128):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    inputs = Input(shape=input_shape[1:])
    
    # Embed the inputs
    embed_english = Embedding(english_vocab_size, english_embedding_size, input_length=input_shape[1])(inputs)
    
    # Bidirectional encoder
    recurrent = GRU(units=output_sequence_length, return_sequences=True, activation=None)
    bidirectional = Bidirectional(recurrent)(embed_english)
    bidirectional = PReLU()(bidirectional)
        
    # Hidden layer
    dense = Dense(hidden_layer_length, activation=None)(bidirectional)
    dense = PReLU()(dense)
    dense = Flatten()(dense)
    dense = RepeatVector(output_sequence_length)(dense)
    
    # Decoder
    recurrent = GRU(units=out_sequence_length, return_sequences=True, activation=None)(dense)
    recurrent = PReLU()(recurrent)
    
    # Output layer
    output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(recurrent)
    
    model = Model(inputs=inputs, outputs=output)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate, clipnorm=1.),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [243]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using model_final
    model = model_final(
        x.shape,
        y.shape[1],
        len(x_tk.word_index),
        len(y_tk.word_index))
    
    print('Network architecture:')
    model.summary()
    
    model.fit(x, y, batch_size=256, epochs=10, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Network architecture:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_225 (InputLayer)       (None, 15)                0         
_________________________________________________________________
embedding_142 (Embedding)    (None, 15, 512)           101888    
_________________________________________________________________
bidirectional_118 (Bidirecti (None, 15, 42)            67284     
_________________________________________________________________
p_re_lu_314 (PReLU)          (None, 15, 42)            630       
_________________________________________________________________
dense_317 (Dense)            (None, 15, 128)           5504      
_________________________________________________________________
p_re_lu_315 (PReLU)          (None, 15, 128)           1920      
_________________________________________________________________
flatten_21 (Flatten)         (None, 1920)             

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.